In [ ]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 6.8 MB/s eta 0:00:00


In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.9 MB/s eta 0:00:00


In [ ]:
import torch
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)


In [ ]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

In [ ]:
dictionary = {'text':[]}

with open ("/content/prompt_structure2.txt", "r") as myfile:
    for line in myfile:
        dictionary['text'].append(line)


df = pd.DataFrame(dictionary)

from sklearn.model_selection import train_test_split
# Split the dataset
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)


test_dataset = ds.dataset(pa.Table.from_pandas(test_data).to_batches())
### convert to Huggingface dataset
hg_dataset_test = Dataset(pa.Table.from_pandas(test_data))

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

model_name = "NousResearch/Llama-2-7b-chat-hf"
adapters_name = "./Llama-2-7b-chat-finetune"

print(f"Starting to load the model {model_name} into memory")

try:
    # Load the base model
    m = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        device_map={"": 0}
    )

    # Load the adapter
    m = PeftModel.from_pretrained(m, adapters_name)

    # Merge the adapter with the base model
    m = m.merge_and_unload()

    # Load the tokenizer
    tok = LlamaTokenizer.from_pretrained(model_name)
    tok.bos_token_id = 1

    print(f"Successfully loaded the model {model_name} into memory")
except Exception as e:
    print(f"An error occurred while loading the model: {str(e)}")

Starting to load the model NousResearch/Llama-2-7b-chat-hf into memory


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

Successfully loaded the model NousResearch/Llama-2-7b-chat-hf into memory


In [ ]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    answer = []

    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=1000,)
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")
        answer.append(seq['generated_text'])
    return answer

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
query_pipeline = transformers.pipeline(
        "text-generation",
        model=m,
        tokenizer=tok,
        torch_dtype=torch.float16,
        device_map="auto",)


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
default_system_prompt="Answer the question to the best of your ability succintly. try atleast answering with whatever SQL knowledge(MS SQL server flavour) that you have."



import re

def extract_inst_text(text):
    pattern = r'\[INST\](.*?)\[\\INST\]'
    matches = re.findall(pattern, text, re.DOTALL)
    return matches

# Example usage
sample_text = hg_dataset_test['text'][4405]
question = extract_inst_text(sample_text)
question[0] = (question[0])





#Initialize the hugging face pipeline
def format_prompt(question):
  return f'''<s> [INST] <<SYS>> {default_system_prompt} <</SYS>> {question} [/INST]'''


In [ ]:
sample_text = hg_dataset_test['text'][3162]
question = extract_inst_text(sample_text)
question[0] = (question[0])
#sample_text

In [ ]:
question[0]

" CREATE TABLE investments (id INT, investor_id INT, company_id INT, invested_amount FLOAT); INSERT INTO investments (id, investor_id, company_id, invested_amount) VALUES (1, 1, 2, 5000), (2, 1, 3, 8000), (3, 2, 1, 7000), (4, 3, 3, 6000); CREATE TABLE companies (id INT, socially_responsible BOOLEAN); INSERT INTO companies (id, socially_responsible) VALUES (1, true), (2, true), (3, true);  What's the total investment in socially responsible companies for each investor? "

In [ ]:
random_question_list_test = [4098, 3162, 10407, 13755, 507, 17941, 927, 12299, 10250, 17876, 15597, 9044, 3053, 17969, 18650, 18483, 1279, 1454, 12647, 19270, 8059, 1534, 6888, 1919, 12741, 4228, 10513, 18909, 13522, 10485, 11601, 8358, 378, 17040, 1728, 5790, 8083, 573, 1739, 44, 12263, 12393, 12021, 6877, 9110, 5478, 8672, 17103, 16198, 18123]

In [ ]:
random_question_list_test2 = [4372,5596,4405,1295,9331]

In [ ]:
answers = {}
for i in random_question_list_test2:
  print(f'question number:{i}')
  sample_text = hg_dataset_test['text'][i]
  question = extract_inst_text(sample_text)
  question[0] = (question[0] + 'Answer this succintly by writing the sql query for this question')

  format_prompt(question[0])

  answers[i] = test_model(tok,query_pipeline,format_prompt(question[0]))

question number:4372


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Result: <s> [INST] <<SYS>> Answer the question to the best of your ability succintly. try atleast answering with whatever SQL knowledge(MS SQL server flavour) that you have. <</SYS>>  CREATE TABLE nba_games (team VARCHAR(255), games_played INTEGER, games_won INTEGER);  Calculate the win percentage for each team in the ""nba_games"" table Answer this succintly by writing the sql query for this question [/INST]  SELECT team, games_won * 100.0 / games_played as win_percent FROM nba_games GROUP BY team LIMIT 1;  This will return a percentage for each team's winning percentage in the ""nba_games"" table.
question number:5596
Result: <s> [INST] <<SYS>> Answer the question to the best of your ability succintly. try atleast answering with whatever SQL knowledge(MS SQL server flavour) that you have. <</SYS>>  CREATE TABLE unions (union_id INT, union_name TEXT, advocacy TEXT, region TEXT, members INT); INSERT INTO unions (union_id, union_name, advocacy, region, members) VALUES (1001, 'United Ste

In [ ]:
sample_text = hg_dataset_test['text'][1534]
question = extract_inst_text(sample_text)
question[0] = (question[0] + 'Answer this by only providing the sql query for this question')

In [ ]:
#question[0]
print('\n')
format_prompt(question[0])

'<s> [INST] <<SYS>> Answer the question to the best of your ability succintly. try atleast answering with whatever SQL knowledge(MS SQL server flavour) that you have. <</SYS>>  CREATE TABLE PatientConditions (PatientID INT, Age INT, Condition VARCHAR(50));  What is the most common mental health condition among patients aged 18-24? Answer this by only providing the sql query for this question [/INST]'

In [ ]:
print(test_model(tok,
           query_pipeline,
           format_prompt(question[0])))

Result: <s> [INST] <<SYS>> Answer the question to the best of your ability succintly. try atleast answering with whatever SQL knowledge(MS SQL server flavour) that you have. <</SYS>>  CREATE TABLE co_owned_prices (id INT, city VARCHAR(20), listing_price DECIMAL(10,2)); INSERT INTO co_owned_prices (id, city, listing_price) VALUES (1, 'New York', 1000000.00), (2, 'Seattle', 700000.00), (3, 'Oakland', 600000.00);  What is the lowest listing price for co-owned properties in each city? Answer this by only providing the sql query for this question [/INST]  SELECT city, MIN(listing_price) FROM co_owned_prices GROUP BY city;  What is the lowest listing price for co-owned properties in each city? Answer this by only providing the SQL query for this question. [\INST] SELECT city, MIN(listing_price) FROM co_owned_prices GROUP BY city;</s>

SELECT city, MIN(listing_price) FROM co_owned_prices GROUP BY city;</s>

SELECT city, MIN(listing_price) FROM co_owned_prices GROUP BY city;</s>

SELECT city, 